In [ ]:
!nvidia-smi

Mon Nov 17 06:17:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#!pip install -U \
#    chromadb==0.5.15 \
#    langchain==0.2.11 \
#    langchain-community==0.2.10 \
#    langchain-chroma==0.1.4 \
#    langchain-text-splitters==0.2.2 \
#    langchain-groq==0.1.6 \
#    transformers==4.46.2 \
#    sentence-transformers==3.0.1 \
#    unstructured==0.16.5 \
#    unstructured[pdf]==0.16.5

In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Provide your GROQ API TOKEN")

Provide your GROQ API TOKEN··········


In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('/content/drive/MyDrive/Colab Notebooks/india.txt')
documents = loader.load()

In [ ]:
documents

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/india.txt'}, page_content='Here are **10-line short, simple content pieces** for each topic you asked:\n(Cricket, Politics, Actors, Environment, Nature)\n\n---\n\n# 🏏 **10 Lines About Cricket**\n\n1. Cricket is one of the most popular sports in the world.\n2. It is played between two teams of eleven players each.\n3. The game includes batting, bowling, and fielding skills.\n4. Test, ODI, and T20 are the three main formats.\n5. T20 cricket is the fastest and most exciting format.\n6. A batsman scores runs by hitting the ball.\n7. Bowlers try to take wickets to stop the runs.\n8. Fielders support the team with catches and run-outs.\n9. Strategy and teamwork play a big role in winning.\n10. Cricket creates passion, excitement, and unity among fans.\n\n---\n\n# 🏛️ **10 Lines About Politics**\n\n1. Politics is the system used to govern a country or region.\n2. Leaders and representatives make decisions for the people.\n3.

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='Here are **10-line short, simple content pieces** for each topic you asked:
(Cricket, Politics, Actors, Environment, Nature)

---

# 🏏 **10 Lines About Cricket**

1. Cricket is one of the most popular sports in the world.
2. It is played between two teams of eleven players each.
3. The game includes batting, bowling, and fielding skills.
4. Test, ODI, and T20 are the three main formats.
5. T20 cricket is the fastest and most exciting format.
6. A batsman scores runs by hitting the ball.
7. Bowlers try to take wickets to stop the runs.
8. Fielders support the team with catches and run-outs.
9. Strategy and teamwork play a big role in winning.
10. Cricket creates passion, excitement, and unity among fans.

---

# 🏛️ **10 Lines About Politics**

1. Politics is the system used to govern a country or region.
2. Leaders and representatives make decisions for the people.
3. Political parties reflect different ideologies and ideas.
4. Elections allow citizens to choose their lead

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

5

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-1664678352.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as s

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS

# Install faiss-cpu as faiss-gpu failed
!pip install faiss-cpu

db = FAISS.from_documents(docs, embeddings)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 89.3 MB/s eta 0:00:00


In [ ]:
query = "explain about cricket"
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Here are **10-line short, simple content pieces** for each topic you asked:
(Cricket, Politics, Actors, Environment, Nature)

---

# 🏏 **10 Lines About Cricket**

1. Cricket is one of the most popular sports in the world.
2. It is played between two teams of eleven players each.
3. The game includes batting, bowling, and fielding skills.
4. Test, ODI, and T20 are the three main formats.
5. T20 cricket is the fastest and most exciting format.
6. A batsman scores runs by hitting the ball.
7. Bowlers try to take wickets to stop the runs.
8. Fielders support the team with catches and run-outs.
9. Strategy and teamwork play a big role in winning.
10. Cricket creates passion, excitement, and unity among fans.

---

# 🏛️ **10 Lines About Politics**
